# Aerial and segmented/original BGT viewer

In [ ]:
import errno
from __future__ import print_function
import glob
from ipywidgets import interact
import ipywidgets as widgets

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import numpy as np
import os
from skimage.io import imread

In [ ]:
# Define data directory
data_dir = os.path.join('/Volumes','Elements','output')

try:
    os.listdir(data_dir)
    
    sample_dirs = sorted(next(os.walk(data_dir))[1])
    aerial_img = '*rial_*.tif*'
    bgt_labels = 'bgt_labels.tif'
    bgt_path = os.path.join('bgt', 'bgt.shp')
    bgt_data = os.path.join('bgt', 'bgt.dbf')
except OSError as e:
    print('Error: Data directory:',data_dir,'not found.')

In [ ]:
# Custom labels and colormap
from matplotlib import colors
import matplotlib as mpl

labelmap = ['undefined',
            'pand',
            'greppel, droge sloot',
            'transitie',
            'waterloop',
            'watervlakte',
            'baan voor vliegverkeer',
            'fietspad',
            'inrit',
            'OV-baan',
            'overweg',
            'parkeervlak',
            'rijbaan autosnelweg',
            'rijbaan autoweg',
            'rijbaan lokale weg',
            'rijbaan regionale weg',
            'ruiterpad',
            'spoorbaan',
            'voetgangersgebied',
            'voetpad',
            'voetpad op trap',
            'woonerf',
            'zee'
           ]

# Colormap based on Visualisatieregels_IMGeo_BGT_standaardvisualisatie.
colors = np.array(
[
    [253,253,254],     # undefined  'xkcd:pale grey',
    [204,0,0],         # pand  'xkcd:reddish brown',
    [201,235,112],     # greppel, droge sloot  'xkcd:powder blue',
    [83,83,83],        # transitie  "xkcd:robin's egg",
    [115,233,255],     # waterloop  'xkcd:azure',
    [190,232,255],     # watervlakte  'xkcd:medium blue',  
    [255,255,255],     # baan voor vliegverkeer  'xkcd:silver',
    [255,255,255],     # fietspad  'xkcd:macaroni and cheese',
    [255,255,255],     # inrit  'xkcd:light blue grey',
    [204,204,204],     # OV-baan  'xkcd:dirty yellow',
    [192,192,192],     # overweg  'xkcd:light grey',
    [255,255,255],     # parkeervlak  'xkcd:grape',
    [153,96,137],      # rijbaan autosnelweg  'xkcd:ivory', # 1
    [230,0,0],         # rijbaan autoweg  'xkcd:puke',
    [255,255,255],     # rijbaan lokale weg  'xkcd:light tan',
    [255,170,0],       # rijbaan regionale weg  'xkcd:buff',
    [255,255,255],     # ruiterpad  'xkcd:pumpkin',
    [192,192,192],     # spoorbaan  'xkcd:charcoal',
    [255,153,153],     # voetgangersgebied  'xkcd:dark sand',
    [255,153,153],     # voetpad  'xkcd:cement',
    [255,153,153],     # voetpad op trap  'xkcd:mushroom',
    [255,255,255],     # woonerf  'xkcd:tan green',
    [153,204,255]      # zee  'xkcd:sea blue'
    ]
)
colors = colors.astype(np.float) / 255.0
cmap =  mpl.colors.ListedColormap(colors, N = len(colors))

## Aerial image and segmented BGT viewer

In [ ]:
plt.close("all")
%matplotlib notebook
fig = plt.figure(figsize=(20, 10))
grid = gridspec.GridSpec(1, 2, figure=fig, width_ratios=[1,1.07])

# Aerial Image
ax1 = fig.add_subplot(grid[0, 0])
ax1.get_xaxis().set_ticks([])
ax1.get_yaxis().set_ticks([])

# BGT Labels Image
ax2 = fig.add_subplot(grid[0, 1], sharex=ax1, sharey=ax1)
ax2.get_xaxis().set_ticks([])
ax2.get_yaxis().set_ticks([])

sample_selector = widgets.Dropdown(
                    options=sample_dirs,
                    value=sample_dirs[0],
                    description='Sample: ',
                    disabled=False,
                    layout=widgets.Layout(margin = '0px 0px 0px 250px'))

bounds=np.arange(0,cmap.N+1)
norm = mpl.colors.BoundaryNorm(bounds, cmap.N)

@interact(sample=sample_selector)
def show_image(sample):
    # Aerial Image
    current_dir = os.path.join(data_dir,sample)
    image = glob.glob(str(current_dir)+os.sep+aerial_img)[0]
    if os.path.exists(image):
        img = imread(image)
        if len(img.shape) == 3:
            if img.shape[2] > 3: # In case of RGBA, only use RGB
                img = img[:,:,0:3]
            ax1.imshow(img)
        else: # In case of grayscale images
            ax1.imshow(img, cmap='gray')
        ax1.set_title(image)
    
    # BGT Labels Image
    labels = imread(os.path.join(data_dir,sample, bgt_labels)).astype(np.uint8)
    ims = ax2.imshow(labels, cmap=cmap)
    ax2.set_title(bgt_labels)
    
    ## Verbose histogram
    #unique, count = np.unique(labels, return_counts=True)
    #print('id'.ljust(10)+'label'.ljust(30)+'occurences')
    #for i in range(len(unique)):
    #    j = unique[i]
    #    print(str(j).ljust(10), str(labelmap[j]).ljust(30), str(count[i]))
    
    # Colorbar
    divider = make_axes_locatable(ax2)
    cbar_ax = divider.append_axes("right", "5%", pad="2%")
    cbar = mpl.colorbar.ColorbarBase(cbar_ax, cmap=cmap, 
                        norm=norm, 
                        ticks=bounds +0.5,
                        spacing='uniform')
    cbar.ax.set_yticklabels(labelmap)
    plt.show()

In [ ]:
# Save examples to disk
nr = 0

In [ ]:
nr += 1
extent = ax1.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
output_dir = os.path.join('/Volumes','Elements')
fig.savefig(os.path.join(output_dir,'example'+str(nr)))